In [46]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [47]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [48]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

### Attempt 1 (remove CLASSIFICATION and increase neurons)

In [49]:
# Look at APPLICATION_TYPE value counts for binning 
application_counts = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than 700
replace_application = list(application_counts[application_counts < 700].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [50]:
# Remove classification column
application_df = application_df.drop(columns=["CLASSIFICATION"])

In [51]:
# Bin AFFILIATION to 3 categories
affiliation_counts = application_df.AFFILIATION.value_counts()
affiliation_counts

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [52]:
replace_affil = list(affiliation_counts[affiliation_counts < 1000].index)

# Replace in dataframe
for aff in replace_affil:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(aff,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [53]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Independent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [54]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(columns=application_cat)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [55]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50, stratify=y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [56]:
X_train_scaled.shape

(25724, 33)

In [83]:
application_df.describe()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
count,34299.000000,3.429900e+04,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,...,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000
mean,0.999854,2.769199e+06,0.532406,0.023441,0.031050,0.788274,0.044958,0.034199,0.035453,0.021138,...,0.021225,0.015831,0.098370,0.006997,0.027843,0.109245,0.004053,0.005394,0.999213,0.000787
std,0.012073,8.713045e+07,0.498956,0.151301,0.173457,0.408538,0.207214,0.181743,0.184924,0.143845,...,0.144136,0.124825,0.297819,0.083358,0.164526,0.311951,0.063532,0.073245,0.028046,0.028046
min,0.000000,5.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,5.000000e+03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,1.000000,5.000000e+03,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,1.000000,7.742000e+03,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,8.597806e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 45

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 90)                3060      
                                                                 
 dense_31 (Dense)            (None, 45)                4095      
                                                                 
 dense_32 (Dense)            (None, 1)                 46        
                                                                 
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "./Resources/weightsOptAttempt1.{epoch:02d}.hdf5"
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 987us/step - loss: 77995.0859 - accuracy: 0.4946
Epoch 2/100
804/804 [==============================] - 1s 978us/step - loss: 89450.1250 - accuracy: 0.4869
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 67543.5312 - accuracy: 0.5007
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 19520.3711 - accuracy: 0.4963
Epoch 5/100
766/804 [===========================>..] - ETA: 0s - loss: 6673.8408 - accuracy: 0.5155
Epoch 5: saving model to ./Resources\weightsOptAttempt1.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 6807.6182 - accuracy: 0.5136
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 34821.8789 - accuracy: 0.5104
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 48493.3047 - accuracy: 0.4862
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 4232.6230 - accuracy: 0.4947
Epoch 

804/804 [==============================] - 1s 998us/step - loss: 0.6912 - accuracy: 0.5324
Epoch 62/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5319
Epoch 63/100
804/804 [==============================] - 1s 982us/step - loss: 0.6914 - accuracy: 0.5324
Epoch 64/100
804/804 [==============================] - 1s 965us/step - loss: 0.6914 - accuracy: 0.5324
Epoch 65/100
765/804 [===========================>..] - ETA: 0s - loss: 0.6913 - accuracy: 0.5309
Epoch 65: saving model to ./Resources\weightsOptAttempt1.65.hdf5
804/804 [==============================] - 1s 994us/step - loss: 0.6913 - accuracy: 0.5307
Epoch 66/100
804/804 [==============================] - 1s 972us/step - loss: 0.6913 - accuracy: 0.5324
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 68/100
804/804 [==============================] - 1s 989us/step - loss: 0.6913 - accuracy: 0.5324
Epoch 69/100
804/804 [=============

In [62]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6669 - accuracy: 0.6943 - 255ms/epoch - 952us/step
Loss: 0.6668626666069031, Accuracy: 0.6943440437316895


In [63]:
# Export our model to HDF5 file
nn.save("AlphabetSoupOptimizedAttempt1.h5")

### Attempt 2 (add another hidden layer)

In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 45
hidden_nodes_layer3 = 15

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 90)                3060      
                                                                 
 dense_34 (Dense)            (None, 45)                4095      
                                                                 
 dense_35 (Dense)            (None, 15)                690       
                                                                 
 dense_36 (Dense)            (None, 1)                 16        
                                                                 
Total params: 7,861
Trainable params: 7,861
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "./Resources/weightsOptAttempt2.{epoch:02d}.hdf5"
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn2.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 17911.3574 - accuracy: 0.4894
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 1866.8961 - accuracy: 0.5012
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6933 - accuracy: 0.5324
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6916 - accuracy: 0.5324
Epoch 5/100
772/804 [===========================>..] - ETA: 0s - loss: 0.6913 - accuracy: 0.5315
Epoch 5: saving model to ./Resources\weightsOptAttempt2.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 9/100
804/804 [===============

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 63/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 65/100
790/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5322
Epoch 65: saving model to ./Resources\weightsOptAttempt2.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 70/100
771/804 [=========================

In [66]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6724 - accuracy: 0.5439 - 360ms/epoch - 1ms/step
Loss: 0.6724036335945129, Accuracy: 0.5439066886901855


In [67]:
# Export our model to HDF5 file
nn2.save("AlphabetSoupOptimizedAttempt2.h5")

### Attempt 3 (change activation function)

In [104]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow.keras.layers import LeakyReLU

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 45
leaky_relu = LeakyReLU(alpha=0.01)
#para_relu = PReLU()

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=leaky_relu))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_59 (Dense)            (None, 90)                3060      
                                                                 
 dense_60 (Dense)            (None, 45)                4095      
                                                                 
 dense_61 (Dense)            (None, 1)                 46        
                                                                 
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [105]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "./Resources/weightsOptAttempt3.{epoch:02d}.hdf5"
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn3.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 52410.0391 - accuracy: 0.4852
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 89811.5781 - accuracy: 0.5007
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 63127.3711 - accuracy: 0.5046
Epoch 4/100
804/804 [==============================] - 1s 987us/step - loss: 51520.0586 - accuracy: 0.5023
Epoch 5/100
762/804 [===========================>..] - ETA: 0s - loss: 45588.4531 - accuracy: 0.4997
Epoch 5: saving model to ./Resources\weightsOptAttempt3.05.hdf5
804/804 [==============================] - 1s 996us/step - loss: 43999.3125 - accuracy: 0.4979
Epoch 6/100
804/804 [==============================] - 1s 992us/step - loss: 65150.6055 - accuracy: 0.4923
Epoch 7/100
804/804 [==============================] - 1s 979us/step - loss: 20206.0488 - accuracy: 0.5044
Epoch 8/100
804/804 [==============================] - 1s 989us/step - loss: 56687.3398 - accuracy: 0.50

Epoch 61/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 62/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 63/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 65/100
772/804 [===========================>..] - ETA: 0s - loss: 0.6912 - accuracy: 0.5314
Epoch 65: saving model to ./Resources\weightsOptAttempt3.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 67/100
804/804 [==============================] - 1s 993us/step - loss: 0.6911 - accuracy: 0.5324
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 69/100
804/804 [==========

In [106]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 2.6434 - accuracy: 0.6921 - 252ms/epoch - 942us/step
Loss: 2.6434426307678223, Accuracy: 0.6921283006668091


In [97]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupOptimizedAttempt3.h5")